In [5]:
import json
import os
import os.path
import re
import time
import traceback
import warnings
from datetime import date

import chromedriver_autoinstaller
import pandas as pd
import requests
import sqlite3
import urllib3
from colorama import init, Fore, Back
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import openpyxl

warnings.filterwarnings("ignore", category=DeprecationWarning)
urllib3.disable_warnings()
requests.packages.urllib3.disable_warnings()
chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

init()

service = Service(executable_path=chromedriver_autoinstaller.install())
driver = webdriver.Chrome(options=options, service=service)
driver.maximize_window()

In [12]:
OL_LOGADO = '23150521 - SERVIÇO DE CENTRALIZAÇÃO DA ANÁLISE DE RECONHECIMENTO DE DIREITOS SRNCO'
timeout = 30

In [42]:
caminho_arquivo = 'protocolos.xlsx'
workbook = openpyxl.load_workbook(caminho_arquivo)
worksheet = workbook['Plan1']

In [51]:
linha = 2

In [52]:
while True:
    try:
        protocolo = worksheet.cell(row=linha, column=2).value
        status_coluna3 = worksheet.cell(row=linha, column=3).value
        id = worksheet.cell(row=linha, column=1).value
        
        if not protocolo:
            input("Finalizado!")
            break

        if status_coluna3:
            print(f"ID {id} já processado, pulando...")
            linha += 1
            continue
        
        print("======================================================================")
        print(f"Iniciando processamento do Protocolo:id Nº{id} - {protocolo}")
        id = worksheet.cell(row=linha, column=1).value
        driver.get("https://esisrec.inss.gov.br/esisrec/pages/painel_de_entrada/consultar_painel_de_entrada.xhtml")
        ActionChains(driver).send_keys(Keys.HOME).perform()
        print("Busca Processo...")
        driver.find_element(By.ID, "formBuscaRapida:buscaRapida").clear()                
        driver.find_element(By.ID, "formBuscaRapida:buscaRapida").send_keys(protocolo)
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        driver.find_element(By.ID, "formBuscaRapida:btBuscaRapida").click()
        time.sleep(2)
        e_nb = driver.find_element(By.ID, "formProcesso:numeroIdentificacao").get_attribute("value")
        unidade = driver.find_element(By.ID, "formProcesso:orgaoAtual").text
        driver.execute_script("window.scrollTo(0, 1000)")
        print("Clicar em Analisar...")
        element = driver.find_element(By.ID, "formProcesso:analisarAnexarDocumentos")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", element)
        time.sleep(1)
        driver.execute_script("document.getElementById('formProcesso:analisarAnexarDocumentos').click();")
        time.sleep(2)
        print("Lançar Evento...")
        driver.find_element(By.XPATH,"//span[@id='iptEventoLancado']/button").click()
        time.sleep(1)
        print("Selecionar Evento Diligencia não cumprida...")
        driver.find_element(By.XPATH,"//span[@id='iptEventoLancado_panel']/ul/li[28]").click()
        data_atual = date.today()
        data_formatada = data_atual.strftime("%d/%m/%Y")
        driver.execute_script("scroll(0,500)")
        time.sleep(1)
        driver.find_element(By.LINK_TEXT, "Digitar um Documento").click()
        time.sleep(1)
        driver.find_element(By.ID, "anexar_docs:tipoDocDigitarDocumentos_label").click()                  
        time.sleep(1)
        print("Selecionar Documento dipo DESPACHO...")
        driver.find_element(By.ID, "anexar_docs:tipoDocDigitarDocumentos_40").click()
        time.sleep(5)
        print("Redigir Despacho...")
        driver.find_element(By.XPATH, "//div[@id='anexar_docs:textEditor_editor']/div").clear()
        driver.find_element(By.XPATH, "//div[@id='anexar_docs:textEditor_editor']/div").send_keys(f'\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t{data_formatada}\n\n\nProtocolo:{protocolo}\nRecorrente: INSTITUTO NACIONAL DO SEGURO SOCIAL – INSS\n\n\n\tDevolução do processo de recurso conforme solicitação do órgão julgador.')
        time.sleep(1)
        print("Anexar Documento...")
        driver.execute_script("document.getElementById('anexar_docs:digitar_doc_anexar').click();")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 1000)")
        print("Movimentar em Bloco...")
        driver.execute_script("document.getElementById('btnMovimentarEmBloco').click();")
        time.sleep(1)
        print("Aguardar retorno...")
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span")))
        time.sleep(1)
        if driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span").text != "Processo movimentado com sucesso.":
            print("Retorno falhou tentar movimentar novamente...")
            driver.execute_script("document.getElementById('btnMovimentarEmBloco').click();")
        time.sleep(1)
        print("Encaminhamento bem sucedido salvar informações na planilha...")
        worksheet.cell(row=linha, column=3).value  = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span").text
        worksheet.cell(row=linha, column=4).value  = driver.find_element(By.XPATH, "/html/body/div[3]/div/form/div/div[1]/div[1]/div/div[1]/div[6]/div[1]/div/span").text
        elemento1 = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div/ul/li/span').text
        elemento2 = driver.find_element(By.XPATH, '/html/body/div[3]/div/form/div/div[1]/div[1]/div/div[1]/div[6]/div[1]/div/span').text
        print(f"{elemento1} - {elemento2}")
        linha += 1
        workbook.save(caminho_arquivo)

    except Exception as e:
        print("\nOcorreu um erro durante a execução:")
        print(f"Erro: {str(e)}")
        print(f"Detalhes do erro:\n{traceback.format_exc()}")
        input("\nPressione ENTER para continuar...")
        # Pega apenas a primeira linha da mensagem de erro ou os primeiros 100 caracteres
        erro_resumido = str(e).split('\n')[0][:100]
        worksheet.cell(row=linha, column=3).value = f"ERRO: {erro_resumido}"
        workbook.save(caminho_arquivo)
        linha += 1
        continue

ID 1 já processado, pulando...
ID 2 já processado, pulando...
ID 3 já processado, pulando...
ID 4 já processado, pulando...
Finalizado!


In [47]:
driver.execute_script("document.getElementById('formProcesso:analisarAnexarDocumentos').click();")

In [ ]:
while Finaliza:
    
    print()
    motivo = ""
    cursor.execute(f"SELECT * FROM dados_excel LIMIT 1 OFFSET {linha}")
    row = cursor.fetchone()

    protocolar="Sim"
    razao = ""

    conferir_concessorio = False
    conferir_contrarrazoes = False
    conferir_info_sub = False
    conferir_juntada_info_prev = False
    tipodoc = 'NB'

    id,protocolo,Espécie,objeto,cpf,cod_org_atual,org_atual,cod_aps,tarefa,originaria,situacao,leitura = row     

    cursor.execute("SELECT [leitura] FROM dados_excel WHERE ID=?", (id,))
    result = cursor.fetchone()
    result = result[0]

    if result == "lido":            
        linha += 1
        continue

    print(id,protocolo)


    cursor.execute(f"UPDATE dados_excel SET 'leitura'='lido' WHERE ID='{id}'")
    conn.commit()


    link_ativo = driver.current_url
    driver.get(link_ativo)


    cursor.execute(f"UPDATE dados_excel SET 'situacao'='{unidade}' WHERE ID='{id}'")
    conn.commit()


    if unidade == 'CRPS - Conselho de Recursos da Previdência Social':
        linha+=1
        continue            
    elif unidade != OL_LOGADO:
        cursor.execute(f"UPDATE dados_excel SET 'situacao'='{unidade}' WHERE ID='{id}'")
        conn.commit()
        linha+=1
        continue

    for i in range(1, 30):
        try:
            element = driver.find_element(By.XPATH, f"/html/body/div[3]/div/form/div/div[1]/div[16]/div/div/div/div[1]/table/tbody/tr[{i}]/td[3]")
            teste = element.text

            print(teste)

            if 'Arquivado' in teste:
                situacao = 'Arquivado'
            if 'Documentação do processo de origem' in teste:
                conferir_concessorio = True
            if 'Contrarrazões do INSS' in teste:
                conferir_contrarrazoes = True
            print("Situação:",situacao)
        except:
            print("Erro...")
            break
    
    #BLOCO 2================================================================================================================
    #busca recurso e verifica se está arquivado
   
while True:
        try:
            ActionChains(driver).send_keys(Keys.HOME).perform()
            driver.find_element(By.ID, "formBuscaRapida:buscaRapida").clear()                
            driver.find_element(By.ID, "formBuscaRapida:buscaRapida").send_keys(protocolo)
            WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            driver.find_element(By.ID, "formBuscaRapida:btBuscaRapida").click()
            time.sleep(2)
            e_nb = driver.find_element(By.ID, "formProcesso:numeroIdentificacao").get_attribute("value")
            unidade = driver.find_element(By.ID, "formProcesso:orgaoAtual").text
            break
        except: 
            time.sleep(1)
            print("Erro Bloco 2 - linha 48")
            linha+=1
            continue

    if situacao == 'Arquivado':
        print(situacao)
        cursor.execute(f"UPDATE dados_excel SET 'situacao'='arquivado' WHERE ID='{id}'")
        conn.commit()            
        linha += 1
        continue            


    #Aguarda documentos==================================================================================================
    contador = 1
    tentativas = 0
    #print("inicio espera")
    while contador < 20:
        try:            
            if len(driver.find_elements(By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span")) == 1:
                element = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span")
                if element.text == "Foi(Foram) incluído(s) novo(s) documento(s) na tarefa de Recurso Ordinário. Para ser(em) incluído (s) selecione a opção \"Sincronizar novos documentos\".":
                    driver.execute_script("scroll(0,1000)")
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formProcesso:baixarDocumentosPendentesGet').click()")
                    time.sleep(2)
                    driver.execute_script("document.getElementById('formProcesso:confirmSim').click()")
                    time.sleep(2)
                    break
            else:
                time.sleep(0.2)
                contador += 1
        except NoSuchElementException:
            break

        tentativas += 1
        if tentativas >= 100:
            break
    #print("fim espera")
    #Aguarda documentos==================================================================================================

    #BLOCO 3==============================================================================================================
    if protocolar == "Sim":
        teste = driver.find_element(By.ID, "formProcesso:tipoIdentificacao:3").get_attribute("checked")
        #print(teste)
    if teste:
        especie = "Seguro Defeso"

    teste = driver.find_element(By.ID, "formProcesso:tipoIdentificacao:3").get_attribute("checked")
    #print(teste)
    if teste:
        especie = "Seguro Defeso"
    time.sleep(5)
    if len(driver.find_elements(By.ID, "formProcesso:optEspecie")) > 0:
        especie = driver.find_element(By.XPATH,"/html/body/div[3]/div/form/div/div[1]/div[1]/div/div[1]/div[2]/div/div[2]/div[2]/div/div/p/span").text
        #print(f"especie:{especie}")
    if len(driver.find_elements(By.ID, "formProcesso:correspondencia_label")) > 0:
        end = driver.find_element(By.ID, "formProcesso:correspondencia_label").text
        #print(f"endereço:{end}")
        endereco = re.sub("'", "", end)

    if endereco == "" or endereco == " ":
        endereco = "Sem endereço."

    teste = driver.find_element(By.ID, "formProcesso:tipoIdentificacao:2").get_attribute("checked")
    #print(teste)
    if teste == "true":
        especie = "NUP"

    teste = driver.find_element(By.ID, "formProcesso:tipoIdentificacao:1").get_attribute("checked")
    #print(teste)
    if teste == "true":
        especie = "CTC"

    if len(driver.find_elements(By.ID, "formProcesso:optEspecie")) == 0 and especie == "":
        especie = 'Não possui espécie.'
        #print(endereco)
        driver.execute_script("window.scrollTo(0, 1000)")
        #print(f"Bloco3:{protocolar}, motivo:{razao}")
    else:        
        #print(f"Bloco3:{protocolar}, motivo:{razao}")
        pass


    #BLOCO 4 ==============================================
    if protocolar == "Sim":
        if len(driver.find_elements(By.ID, "formProcesso:encaminharProcesso")) == 0:
            situacao = "Não permite encaminhamento, verificar se o processo encontra-se arquivado ou no CRPS."
            #protocolar="Não"
            print("parou")
            print("Erro")
            cursor.execute(f"UPDATE dados_excel SET 'leitura'='erro' WHERE ID='{id}'")
            conn.commit()
            continue
            
    actions = ActionChains(driver)
    time.sleep(1)
    actions.move_to_element(element).perform()
    time.sleep(1)
    element = driver.find_element(By.CSS_SELECTOR, "body")
    time.sleep(1)
    actions = ActionChains(driver)
    time.sleep(1)
    actions.move_to_element(element, 0, 0).perform()
    time.sleep(1)
    driver.find_element(By.ID, "formProcesso:motivo1").click()
    time.sleep(3)
    driver.find_element(By.CSS_SELECTOR, ".ui-state-hover > .ui-icon").click()
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,0)")
    time.sleep(3)
    driver.find_element(By.ID, "formProcesso:motivo1_381").click()
    time.sleep(3)            

    #BLOCO 5 ==================================================
    if protocolar == "Sim":
        motivo = driver.find_element(By.ID, "formProcesso:motivo1_label").text
        print(f"motivo:",motivo)
        if motivo == " " or motivo == "":            
            motivo = "FALTA MOTIVO"

            if especie == 'Seguro Defeso':
                for i in range (1,100):

                    motivo = driver.find_element(By.ID, "formProcesso:motivo1_label").text
                    if motivo == "":
                        try:
                            time.sleep(1)
                            driver.find_element(By.ID, "formProcesso:motivo1_label").click()
                            time.sleep(5)                    
                            driver.find_element(By.ID, "formProcesso:motivo1_filter").send_keys("DIVERSOS")
                            time.sleep(1)                
                            driver.find_element(By.ID, "formProcesso:motivo1_101").click()
                            time.sleep(1) 
                            driver.execute_script("document.getElementById('formProcesso:btSalvar').click();")
                        except:

    #                     if len(driver.find_elements(By.XPATH, "/html/body/div[3]/div/form/div/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/span")) > 0:
    #                         driver.execute_script("document.getElementById('formProcesso:j_idt131').click();")

                            time.sleep(1)

                if len(driver.find_elements(By.XPATH, "/html/body/div[4]/div/div[2]/div/ul/li/span")) > 0:
                    if driver.find_element(By.XPATH,"/html/body/div[4]/div/div[2]/div/ul/li/span").text == "Correspondência é um campo obrigatório.":
                        if len(driver.find_elements(By.ID, "formProcesso:correspondencia_label")) > 0:
                            driver.execute_script("document.getElementById('formProcesso:correspondencia_label').click();")
                        if len(driver.find_elements(By.ID, "formProcesso:correspondencia_1")) > 0:
                            driver.execute_script("document.getElementById('formProcesso:correspondencia_1').click();")
                        else:
                            endereco = "Correspondência - O campo CEP é um campo obrigatório."




        if len(driver.find_elements(By.ID, "formProcesso:btConsultaSub")) > 0:
            driver.execute_script("document.getElementById('formProcesso:btConsultaSub').click();")
            time.sleep(1)


        time.sleep(1)
        if len(driver.find_elements(By.ID, "formDadosSub:btnAtualizarMotivo")) > 0:
            driver.execute_script("document.getElementById('formDadosSub:btnAtualizarMotivo').click();")

        time.sleep(1)


        if especie != 'Seguro Defeso':
            if conferir_info_sub == False:
                if len(driver.find_elements(By.ID, "formDadosSub:btnIncluirDocumentacao")) > 0:
                    driver.execute_script("document.getElementById('formDadosSub:btnIncluirDocumentacao').click();")
                else:
                    driver.execute_script("document.getElementById('formDadosSub:btnIncluirDocumentacaoConfirmPopup').click();")
                    if len(driver.find_elements(By.ID, "dialogConfirmIncluirSub_title")) > 0:
                        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        else:

            driver.execute_script("document.getElementById('formDadosSub:btnConsultaSubVoltar').click();")



    #BLOCO 6 ==================================================
    if protocolar == "Sim":
        element_text = driver.find_element(By.ID, "formProcesso:correspondencia_label").text
        #print(element_text)

        if element_text == "" or element_text == " ":
            if len(driver.find_elements(By.ID, "formProcesso:correspondencia_label")) > 0:
                driver.execute_script("document.getElementById('formProcesso:correspondencia_label').click();")
            if len(driver.find_elements(By.ID, "formProcesso:correspondencia_1")) > 0:
                driver.execute_script("document.getElementById('formProcesso:correspondencia_1').click();")

    else:
        #print(f"Bloco6:{protocolar}, motivo:{razao}")
        pass

    #BLOCO 7 ==================================================
    #ajustar endereço
    if protocolar == "Sim":
        teste = ""
        driver.execute_script("document.getElementById('formProcesso:btSalvar').click();")
        time.sleep(1)
        teste = driver.find_element(By.XPATH, "//span[contains(@class,'ui-messages-error-summary')]").text
        if teste != '':                
            while Finaliza:
                try:
                    driver.get(link_ativo)
                    time.sleep(5)
                    driver.execute_script("document.getElementById('formProcesso:btObterEnderecos').click();")
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formDadosEnderecos:pessoa_label').click();")
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formDadosEnderecos:pessoa_1').click();")
                    time.sleep(5)
                    #driver.find_element(By.XPATH, "//button[contains(@id,'formEnderecoNaoEncontrado:')]").send_keys(Keys.RETURN)
                    #time.sleep(1)
                    #driver.execute_script("document.getElementById('formDadosEnderecos:btnVoltarEnderecos').click();")
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formDadosEnderecos:endCorrespondencia_label').click();")
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formDadosEnderecos:endCorrespondencia_1').click();")
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formDadosEnderecos:btnSalvarEnderecos').click();")
                    time.sleep(1)
                    #driver.execute_script("document.getElementById('formDadosEnderecos:btnMsgEnderecos').click();")
                    time.sleep(1)
                    break
                except:
                    print("Erro ao puxar endereço...")
                    break


        teste = driver.find_element(By.XPATH, "//span[contains(@class,'ui-messages-error-summary')]").text
        if teste != '':                
            razao = "Correspondência - O campo País é um campo obrigatório."            
        driver.execute_script("document.getElementById('formProcesso:btSalvar').click();")
        time.sleep(2)
        if driver.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div/ul/li[1]/span").text != "Processo alterado com sucesso.":                                            
            erro_endereco = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div/ul/li[1]/span").text
            cursor.execute(f"UPDATE dados_excel SET 'tarefa'='{erro_endereco}' WHERE ID='{id}'")
            conn.commit()
            print("problema endereço")
            linha+=1
            continue
    else:
        #print(f"Bloco7:{protocolar}, motivo:{razao}")
        pass

    #BLOCO 8 ==================================================
    if protocolar == "Sim":
        driver.find_element(By.ID, "formProcesso:correspondencia_label").text

        if teste != "":    
            endereco = "Sem endereço."
            razao = "Sem endereço."


        if len(driver.find_elements(By.ID, "formProcesso:correspondencia_label")) > 0:
            teste = driver.find_element(By.ID, "formProcesso:correspondencia_label").text
            if teste == '':
                endereco = "Sem endereco"
                razao = "Sem endereço."


        if not conferir_juntada_info_prev:
            if especie == "Auxílio-doença previdenciário" or especie == "Auxílio-doença por acidente do trabalho":
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarDocumentosDoCentralService').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarDadosCadastraisCNIS').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarDeclaracaoBeneficios').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarRemuneracoes').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarLaudoSABI').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarCartaConsessao').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:btSalvarAnexarDocumentosCentralService').click();")
                time.sleep(10)
                driver.find_element(By.ID, "formProcesso:j_idt442").click()
                time.sleep(2)
        #print(f"Bloco8:{protocolar}, motivo:{razao}")
    else:
        #print(f"Bloco8:{protocolar}, motivo:{razao}")
        pass






    #BLOCO 9 ==================================================
    if protocolar == "Sim":        
        teste = driver.find_element(By.ID, "formProcesso:correspondencia_label").text
        if teste == "":
            endereco = "Sem endereço."
            razao = "Sem endereço."                


        if len(driver.find_elements(By.ID, "formProcesso:correspondencia_label")) > 0:
            teste = driver.find_element(By.ID, "formProcesso:correspondencia_label").text
            if teste == '':
                endereco = "Sem endereco"
                razao = "Sem endereço."


        if not conferir_juntada_info_prev:
            if especie == "Auxílio-doença previdenciário" or especie == "Auxílio-doença por acidente do trabalho":
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarDocumentosDoCentralService').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarDadosCadastraisCNIS').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarDeclaracaoBeneficios').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarRemuneracoes').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarLaudoSABI').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:anexarCartaConsessao').click();")
                time.sleep(1)
                driver.execute_script("document.getElementById('formProcesso:btSalvarAnexarDocumentosCentralService').click();")
                time.sleep(10)
                driver.find_element(By.ID, "formProcesso:j_idt442").click()
                time.sleep(2)
        #print(f"Bloco9:{protocolar}, motivo:{razao}")
    else:
        #print(f"Bloco9:{protocolar}, motivo:{razao}")
        pass

    #BLOCO 10 ==============
    #anexar originário
    if protocolar == "Sim":
        if not conferir_concessorio:
            if especie == "Auxílio-doença previdenciário":
                time.sleep(1)
            if especie != "Auxílio-doença previdenciário" and especie != "Auxílio-doença por acidente do trabalho":
                if originaria != "":
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formProcesso:incluirDocumentoConcessao').click();")
                    time.sleep(1)
                    element = driver.find_element(By.ID, "formProcesso:novoValorProtocoloTarefa")
                    element.clear()
                    time.sleep(1)
                    element.send_keys(originaria)
                    time.sleep(1)
                    driver.execute_script("document.getElementById('formProcesso:btSalvarProtocoloTarefa').click();")
                    time.sleep(5)
    else:
        pass        
        #print(f"Bloco10:{protocolar}, motivo:{razao}")

    #BLOCO 11 =============================================
    if protocolar == "Sim":
        if conferir_contrarrazoes == False:
            print("Não possui contrarrazões")
        else:
            print("Já possui contrarrazões")
    else:
        pass

     #BLOCO 12 ============================================
    driver.execute_script("document.getElementById('formProcesso:analisarAnexarDocumentos').click();")

    #BLOCO 13 ============================================
    if protocolar == "Sim":
        if len(driver.find_elements(By.XPATH, "/html/body/div[4]/div/div[2]/div/ul/li/span")) > 0:
            if driver.find_element(By.XPATH,"/html/body/div[4]/div/div[2]/div/ul/li/span").text == "Correspondência - O campo CEP é um campo obrigatório.":
                endereco = "Correspondência é um campo obrigatório."
                linha+=1
                continue
    else:
        pass
        #print(f"Bloco13:{protocolar}, motivo:{razao}")    
    #BLOCO 14 ============================================
    if conferir_contrarrazoes == False:        
        if protocolar == "Sim":
            if len(driver.find_elements(By.XPATH, "/html/body/div[4]/div/div[2]/div/ul/li/span")) > 0:
                if driver.find_element(By.XPATH,"/html/body/div[4]/div/div[2]/div/ul/li/span").text == "Correspondência é um campo obrigatório.":
                    driver.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div/ul/li/span").click()
                    driver.find_element(By.XPATH, "/html/body/div[6]/div/ul/li[2]").click()
                    driver.execute_script("document.getElementById('formProcesso:analisarAnexarDocumentos').click();")    
        else:
            #print(f"Bloco14:{protocolar}, motivo:{razao}")
            pass



    #BLOCO 15 ================================================================================
    if protocolar == "Sim":
        protocolo_limpo = protocolo.replace(".", "").replace("/", "").replace("-", "")
        time.sleep(1)
        driver.get(f"https://esisrec.inss.gov.br/esisrec/pages/movimentacao_processual/movimento_processual.xhtml?nupProcesso={protocolo_limpo}")
        time.sleep(5)
        driver.find_element(By.XPATH,"//span[@id='iptEventoLancado']/button").click()
        time.sleep(5)
        driver.find_element(By.XPATH,"//span[@id='iptEventoLancado_panel']/ul/li[18]").click()     

    #BLOCO 15 ================================================================================


    #BLOCO 16 ================================================================================       
    if protocolar == "Sim":            
        while True:
            try:
                data_atual = date.today()
                data_formatada = data_atual.strftime("%d/%m/%Y")
                time.sleep(1)
                driver.execute_script("scroll(0,500)")
                time.sleep(1)
                driver.find_element(By.LINK_TEXT, "Digitar um Documento").click()                    
                time.sleep(1)
                driver.find_element(By.ID, "anexar_docs:tipoDocDigitarDocumentos_label").click()                  
                time.sleep(1)
                driver.find_element(By.ID, "anexar_docs:tipoDocDigitarDocumentos_31").click()
                time.sleep(5)
                driver.find_element(By.XPATH, "//div[@id='anexar_docs:textEditor_editor']/div").clear()
                time.sleep(1)
                driver.find_element(By.XPATH, "//div[@id='anexar_docs:textEditor_editor']/div").send_keys(f'\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tAGÊNCIA DA PREVIDÊNCIA SOCIAL CEAB RECONHECIMENTO DE DIREITO DA SRV,{data_formatada}\n\nNº:{objeto}\nProtocolo:{protocolo}\nRecorrente: INSTITUTO NACIONAL DO SEGURO SOCIAL – INSS\n\n\n\tTrata-se de requerimento de {especie}.\n\tO segurado interpôs Recurso Administrativo, solicitando a concessão do benefício; \n\tEm atenção ao art. 305, parágrafo 7º, do Decreto 3.048/99, o qual estabelece que “as razões do indeferimento e os demais elementos que compõem o processo administrativo previdenciário substituirão as contrarrazões apresentadas pelo INSS”, remetemos o presente recurso ao Conselho de Recursos da Previdência Social – CRPS, servindo como contrarrazões os motivos do indeferimento.\tRegistramos, oportunamente, que nos termos do art. 9º e do item 1, do Anexo I, ambos da Portaria nº 59/DIRBEN/INSS, de 26/11/2019, “é competência do CRPS decidir quanto à tempestividade e à existência de fatos impeditivos para o conhecimento do recurso”, bem como que “a manifestação prévia do requerente acerca da existência ou não de ação judicial com o mesmo pedido do recurso administrativo dispensa a sua convocação”.\n\tAo CRPS.\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRicardo Alves de Oliveira Silva\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tMatrícula 2035843')
                time.sleep(5)
                break
            except:                   
                print("Erro bloco 16")                    
                continue
    else:
        #print(f"Bloco16:{protocolar}, motivo:{razao}")
        pass
    #BLOCO 16 ====================================================================================

    #BLOCO 17 ============================================
    if protocolar == "Sim":
        driver.execute_script("document.getElementById('anexar_docs:digitar_doc_anexar').click();")
        driver.execute_script("window.scrollTo(0, 1000)")
        time.sleep(3)
    else:
        pass
        #print(f"Bloco17:{protocolar}, motivo:{razao}")

    #BLOCO 18 ============================================          
    ActionChains(driver).send_keys(Keys.HOME).perform()
    time.sleep(1)


    driver.execute_script("document.getElementById('btnMovimentarEmBloco').click();")        
    WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span")))

    if driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span").text != "Processo movimentado com sucesso.":
        driver.execute_script("document.getElementById('btnMovimentarEmBloco').click();")            

    #BLOCO 19 ============================================
    if protocolar == "Sim":            
        while True:
            try:
                ActionChains(driver).send_keys(Keys.HOME).perform()
                driver.get(link_ativo)    
                time.sleep(3)

                if len(driver.find_elements(By.XPATH, "/html/body/div[3]/div/div[2]/div/ul/li/span")) == 0:
                    time.sleep(5)
                    driver.execute_script("document.getElementById('formProcesso:encaminharProcesso').click();")
                    time.sleep(10)
                    driver.find_element(By.XPATH, "/html/body/div[3]/div/form/span/div[1]/div/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div/div").click()
                    time.sleep(5)
                    driver.execute_script("document.getElementById('btnEncaminharEmBloco').click();")#erro                    
                    time.sleep(5)
                    break
                else:
                    driver.get(link_ativo)
                    time.sleep(5)
                    driver.execute_script("document.getElementById('formProcesso:encaminharProcesso').click();")
                    time.sleep(10)
                    driver.find_element(By.XPATH, "/html/body/div[3]/div/form/span/div[1]/div/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div/div").click()
                    time.sleep(5)
                    driver.execute_script("document.getElementById('btnEncaminharEmBloco').click();")#erro                    
                    time.sleep(5)
                    break
            except NoSuchElementException:
                # Caso ocorra um erro, continue no loop
                print("Erro bloco 19")
                continue            
    else:
        #print(f"Bloco19:{protocolar}, motivo:{razao}")
        pass
        #BLOCO  fim============================================        

    print("Finaliza")
    driver.get(link_ativo)
    time.sleep(2)
    unidade = driver.find_element(By.ID, "formProcesso:orgaoAtual").text
    time.sleep(2)
    cursor.execute(f"UPDATE dados_excel SET 'situacao'='{unidade}' WHERE ID='{id}'")        
    conn.commit()        
    linha +=1